In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [17]:
import numpy as np
import pandas as pd
import warnings
import time
import sys
from IPython.display import display, HTML
import tabulate #sudo pip3 install tabulate
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)
def update_column(ds,column,update=False):
    update_column=True
    if column in ds:
        if ds[column] and not update: #it update=True willreplace value
            update_column=False
    return update_column

def _get_doi(surname='Florez',\
        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other=''):
        '''
        Search doi from http://search.crossref.org/ 
        '''
        import re
        import requests
        doi={}
        search=''
        if surname:
            search=surname
        if title:
            if len(search)>0:
                search=search+', '+title
        if other:
            if len(search)>0:
                search=search+', '+other
                
        r=requests.get('http://search.crossref.org/?q=%s' %search)
        urldoi='http://dx.doi.org/'
        doitmp=''
        if len(r.text.split(urldoi))>1:
            doitmp=r.text.split(urldoi)[1].split("\'>")[0].replace('&lt;','<').replace('&gt;','>')
        #check doi is right by searching for all words in doi -> output title
        if doitmp:
            json='https://api.crossref.org/v1/works/'
            rr=requests.get( json+urldoi+doitmp )
            if rr.status_code==200:
                if 'message' in rr.json():
                    chktitle = re.sub(r"\$.*?\$","",title) # better remove all math expressions
                    chktitle = re.sub(r"[^a-zA-Z0-9 ]", " ", chktitle).split(' ')
                    if chktitle:
                        if not -1 in [(rr.json()["message"]['title'][0]).find(w)  for w in chktitle]:
                            doi=rr.json()["message"]
                        
        return doi
    
def _get_IF(issn):
    print(issn)
    
class publications(object):
    '''Add Generic publication data'''
    pass
class articles(publications):
    '''Read csv file exported by Google Scholar Citations profile and
       automatically add informations about:
       DOI: from title and author
       Journal title: from DOI
       ISSN of Jornal
       Impact factor of journal
       Institution_Authors: requires Data Base
       '''
    journal=pd.Series()
    columns=pd.Series({'Full_Name':'Full_Name','Author_Names':'Author_Names','Control':'Control',\
                      'Institution_Authors':'Institution_Authors','Institution_Group':'Institution_Group'})

    def __init__(self,csv_file='citations.csv',citations_file=None,authors_file=None,group_file=None):
        #DEBUG: check file
        self.articles=pd.read_csv(csv_file).fillna('')
        #Fix problem with column Authors
        if self.articles.shape[0]>0 and self.articles.columns[0].find('Authors')>-1:
            self.articles=self.articles.rename(columns={self.articles.columns[0]:'Authors'})
        if citations_file:
            #DEBUG: check file
            self.citations=pd.read_csv(citations_file).fillna('')
            
        if authors_file:
            #DEBUG: chek is file exists,
            self.institution_authors=pd.read_json(authors_file).fillna('')
            
        if group_file:
            #DEBUG: chek is file exists,
            self.institution_group=pd.read_json(group_file).fillna('')
            
    def add_institution_author(self):
        #DEBUG: Check and load file
        full_name=input('Full name: ')
        #DEBUG: Ask for more author names and update author names separated by semicolons
        author_names=input('Author names\n(Example: Juan Perez; J. Perez; J. Pérez):\n')
        #DEBUG: default to 0
        control=input('Additional identitication number: ') 
        
        #DEBUG: if not file creates it
        iffile=False
        if not iffile:
            self.institution_authors=pd.DataFrame()
            
        self.institution_authors=self.institution_authors.append({self.columns.Full_Name:full_name,\
                                                              self.columns.Author_Names:author_names,\
                                                              self.columns.Control:control},ignore_index=True)
        self.institution_authors.to_json('authors.json')
        
    def add_institution_group(self):
        wrn='Author not found'
        search_query=input('Find author by first name and first surname\n(Example: Juan Pérez);\n')
        sql=search_query.split(' ')
        #DEBUG: low_case
        if len(sql)>0:
            sql_name=sql[0]
            if len(sql)>1:
                sql_surname=sql[1]
            else:
                sql_surname=''
            
            author_match=self.institution_authors[\
                             np.logical_and( self.institution_authors.Full_Name.str.contains(sql_name),\
                                             self.institution_authors.Full_Name.str.contains(sql_surname) )]
            if not author_match.shape[0]:
                author_match=self.institution_authors[\
                             np.logical_or( self.institution_authors.Full_Name.str.contains(sql_name),\
                                            self.institution_authors.Full_Name.str.contains(sql_surname) )]
            if not author_match.shape[0]:
                warnings.warn(wrn)
            else:    
                author_match=author_match.reset_index(drop=True)
                author_match['Number']=author_match.index
                if author_match.shape[0]==1:
                    author_Series=author_match.ix[0]
                    #print( tabulate.tabulate(author_match,headers='keys', tablefmt='psql'))
                else:
                    print( tabulate.tabulate(author_match,headers='keys', tablefmt='psql'))
                    ai=input('Give Number of line:')
                    author_Series=author_match.ix[ai]
                Group=input('Group name for\n%s\n(Example: IA: Inteligencia Artificial)' %author_Series.Full_Name)
                #DEBUG: if not file creates it
                iffile=False
                if not iffile:
                    self.institution_group=pd.DataFrame()  
                    self.institution_group=self.institution_group.append({self.columns.Full_Name:author_Series.Full_Name,\
                                                                  self.columns.Institution_Group:Group},ignore_index=True)
                    self.institution_group.to_json('groups.json')
        else:
            warnings.warn(wrn)

        
        
    def get_doi(self,surname='Florez',\
        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other=''):
        return _get_doi(surname='Florez',\
        title=r'Baryonic violation of R-parity from anomalous $U(1)_H$',other='')
    
    def articles_update(self,cites=True,institution_authors=False,institution_groups=False,DOI=False):
        self.fulldoi=pd.DataFrame()
        newcolumns=['Institution_Authors','Institution_Groups','DOI','ISSN','DOI_Journal']
        for newcolumn in newcolumns:
            if not newcolumn in self.articles.columns:
                self.articles[newcolumn]=''

        self.articles=self.articles.reset_index(drop=True).fillna('')
        print('Updating entry:',end="")
        for i in range(self.articles.shape[0]):
            time.sleep(0.3) #avoid robot detection
            print('%d.' %i,end="")
            #Update Full names: TODO: Unify algorithm
            if not self.articles.ix[i].Institution_Authors:
                institution_authors=True
                
            if self.articles.ix[i].Authors and institution_authors:
                inst_auth=''
                inst_auth_sep=';'
                for a in filter(None, self.articles.ix[0].Authors.replace('; ',';').split(';') ):
                    af=self.institution_authors[self.institution_authors.Author_Names.str.contains(a)].reset_index(drop=True)
                    if af.shape[0]==1:
                        if inst_auth:
                            inst_auth=inst_auth+inst_auth_sep
                        if af.Full_Name.values.shape[0]==1:
                            inst_auth=inst_auth+af.Full_Name.values[0]
                    elif af.shape[0]>1:
                        print('Improve real name matching')
                    if update_column(self.articles.ix[i],'Institution_Authors'):
                        self.articles.loc[i,'Institution_Authors']=inst_auth
            #Update Groups:
            if not self.articles.ix[i].Institution_Groups:
                institution_groups=True
            if self.articles.ix[i].Institution_Authors and institution_groups:
                inst_grp=''
                inst_grp_sep=';'
                chka=self.articles.ix[i].Institution_Authors.split(';')
                if len(chka)>0:
                    for fa in chka:
                        qry=self.institution_group[self.institution_group.Full_Name.str.contains(fa)].reset_index(drop=True)
                        if qry.shape[0]==1:
                            g=qry.Institution_Group.values[0]
                        else:
                            print('DEBUG improve search group for %s' %fa)
                        if inst_grp:
                            inst_grp=inst_grp+inst_grp_sep
                        
                        inst_grp=inst_grp+g
                    
                if update_column(self.articles.ix[i],'Institution_Groups'):
                        self.articles.loc[i,'Institution_Groups']=inst_grp
                    
                
            #Update DOI:
            if not self.articles.ix[i].DOI:
                rr=_get_doi(surname=self.articles.ix[i].Authors.split(';')[0].split(',')[0],\
                            title=self.articles.ix[i].Title)
                if len(rr)>0:
                    rr=pd.Series(rr)
                    if update_column(self.articles.ix[i],'DOI') and 'URL' in rr:
                        self.articles.loc[i,'DOI']=rr['URL']
                    if update_column(self.articles.ix[i],'ISSN') and 'ISSN' in rr:
                        issn=''
                        if type(rr.ISSN)==list:
                            issn=';'.join(rr.ISSN)
                        else:
                            print('DEBUG: Improve DOI -> ISSN not a list')
                        self.articles.loc[i,'ISSN']=issn
                    if update_column(self.articles.ix[i],'DOI_Journal') and 'container-title' in rr:                        
                        journal=[ j for j in rr['container-title'] if j.find('.')<0] 
                        if len(journal)==1:
                            self.articles.loc[i,'DOI_Journal']=journal[0]
                        else:
                            print('DEBUG: Improve DOI_journal have dots')
                        
                        
                self.fulldoi=self.fulldoi.append(rr,ignore_index=True)
            
        return self.fulldoi.fillna('')

    def to_csv(self,csvfile):
        self.articles.to_csv(csvfile,index=False)
        if self.fulldoi.shape[0]>0:
            self.fulldoi.to_json('fulldoi.json')
        
                        
            

Here we assume that there exist a data base with the authors multiple names associated to the real name in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Author_Names</th>
      <th>Control</th>
      <th>Full_Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Zapata, Oscar;Zapata, O.</td>
      <td> 0</td>
      <td> Óscar Alberto Zapata Noreña</td>
    </tr>
  </tbody>
</table>
Where control is some identification number associated to the author.


We assume also that there is a data base with the Group at which some real name author belongs in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Full_Name</th>
      <th>Institution_Group</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Óscar Alberto Zapata Noreña</td>
      <td> GFIF</td>
    </tr>
  </tbody>
</table>

In [18]:
a=articles('citations.csv',authors_file='authors.json',group_file='groups.json')
a.institution_authors

,Author_Names,Control,Full_Name
0,"Zapata, Oscar;Zapata, O.",0,Óscar Alberto Zapata Noreña


In [19]:
a.institution_group

,Full_Name,Institution_Group
0,Óscar Alberto Zapata Noreña,GFIF


In [20]:
fulldoi=a.articles_update()

Updating entry:0.1.2.

In [21]:
a.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics


In [8]:
fulldoi[:1]

,DOI,ISSN,URL,article-number,author,container-title,created,deposited,indexed,issue,issued,license,link,member,prefix,published-online,publisher,reference-count,score,source,subject,subtitle,title,type,volume,page
0,10.1103/physrevd.79.013011,"[1550-7998, 1550-2368]",http://dx.doi.org/10.1103/physrevd.79.013011,013011,"[{'affiliation': [], 'given': 'D.', 'family': 'Aristizabal Sierra'}, {'affiliation': [], 'given': 'Jisuke', 'family': 'Kubo'}, {'affiliation': [], 'given': 'Daijiro', 'family': 'Suematsu'}, {'affi...","[Physical Review D, Phys. Rev. D]","{'date-parts': [[2009, 1, 28]], 'timestamp': 1233162501000, 'date-time': '2009-01-28T17:08:21Z'}","{'date-parts': [[2015, 3, 26]], 'timestamp': 1427391551000, 'date-time': '2015-03-26T17:39:11Z'}","{'date-parts': [[2015, 12, 20]], 'timestamp': 1450639904869, 'date-time': '2015-12-20T19:31:44Z'}",1,"{'date-parts': [[2009, 1, 28]]}","[{'content-version': 'vor', 'URL': 'http://link.aps.org/licenses/aps-default-license', 'start': {'date-parts': [[2009, 1, 28]], 'timestamp': 1233100800000, 'date-time': '2009-01-28T00:00:00Z'}, 'd...","[{'intended-application': 'syndication', 'content-version': 'vor', 'URL': 'http://link.aps.org/article/10.1103/PhysRevD.79.013011', 'content-type': 'unspecified'}]",http://id.crossref.org/member/16,http://id.crossref.org/prefix/10.1103,"{'date-parts': [[2009, 1, 28]]}",American Physical Society (APS),66,1,CrossRef,[Nuclear and High Energy Physics],[],"[Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals]",journal-article,79,


## Save uptated data base

In [9]:
a.to_csv('newcitations.csv')

Restore saved data base

In [10]:
arts=articles('newcitations.csv',authors_file='authors.json',group_file='groups.json')

In [11]:
arts.articles_update()

Updating entry:0.1.2.

""


In [12]:
arts.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics


In [52]:
rr=fulldoi.ix[1]
rr

DOI                                                                                                                                                                                              10.1103/physrevd.85.023523
ISSN                                                                                                                                                                                                 [1550-7998, 1550-2368]
URL                                                                                                                                                                            http://dx.doi.org/10.1103/physrevd.85.023523
article-number                                                                                                                                                                                                       023523
author              [{'affiliation': [], 'family': 'Restrepo', 'given': 'Diego'}, {'affiliation': [], 'family': 'Taoso',

In [63]:
rr.ISSN

['1550-7998', '1550-2368']

In [51]:
issn

'1475-7516'

In [42]:
issn

'1550-7998;1550-2368'